# Analysis for Figure2c

In [8]:
1

1

In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["font.family"] = "DeJavu Serif"
plt.rcParams["font.serif"] = ["Times New Roman"]

In [3]:
import copy
import gc
import os
import sys
import warnings
import time
import importlib
import subprocess

import anndata as ad
import networkx as nx
import numpy as np
import pandas as pd
import scanpy as sc
import torch

sys.path.append("/home/icb/kemal.inecik/work/codes/tardis")
import tardis

tardis.config = tardis.config_server
sc.settings.verbosity = 3

print(f"CUDA used: {torch.cuda.is_available()}")

CUDA used: True


In [4]:
n_reserved_latent = 8
n_unreserved_latent = 24

dataset_handles = {
    "Kanemaru": {"models": {
        "scvi": {'kanemaru': dict()}, 
        "tardis": {
            "kanemaru_sex_age_unit": {
                "targets": ["age", "sex", "integration_biological_unit"] # order is important
            }
        }, 
    }}, 
    "Braun": {"models": {
        "scvi": {'braun': dict()}, 
        "tardis": {
            "braun_sex_age": {
                "targets": ["age", "sex"]  # order is important
            }
        }, 
    }},
    "Suo": {"models": {
        "scvi": {'suo': dict()},
        "tardis": {
            "suo_age": {"targets": ["age"]},  # order is important
            "suo_sex": {"targets": ["sex"]},  # order is important
            "suo_organ": {"targets": ["organ"]},  # order is important
            "suo_platform": {"targets": ["platform"]},  # order is important
            "suo_sex_age_organ_platform": {"targets": ["age", "sex", "integration_library_platform_coarse", "organ"]},  # order is important
        }, 
    }},
    "Miller": {"models": {
        "scvi": {'miller': dict()}, 
        "tardis": {
            "miller_age": {"targets": ["age"]}, # order is important
            "miller_sex": {"targets": ["sex"]}, # order is important
            "miller_sex_age": {"targets": ["age", "sex"]}, # order is important
        }, 
    }},
    "He": {"models": {
        "scvi": {'he': dict()}, 
        "tardis": {
            "he_sex_age": {"targets": ["age", "sex"]} # order is important
        }, 
    }},
    "Garcia": {"models": {
        "scvi": {'garcia': dict()}, 
        "tardis": {
            "garcia_sex_age_status": {"targets": ["age", "sex", "integration_sample_status"]} # order is important
        }, 
        
    }},
}

In [5]:
k_neighbours = [15, 30]
for handle in dataset_handles:
    adata_dir = os.path.join(tardis.config.io_directories["processed"], f"dataset_complete_{handle}.h5ad")
    dataset_handles[handle]["adata_dir"] = adata_dir
    for model_type in dataset_handles[handle]["models"]:
        for model_name in dataset_handles[handle]["models"][model_type]:
            model_dir = os.path.join(tardis.config.io_directories["models"], model_name)
            dataset_handles[handle]["models"][model_type][model_name]["model_dir"] = model_dir
            dataset_handles[handle]["models"][model_type][model_name]["latent_dir"] = {i: dict() for i in k_neighbours}
            for n_neighbors in k_neighbours:
                latent_dir = os.path.join(tardis.config.io_directories["temp"], "latent", f"{model_name}_{model_type}_{n_neighbors}_latent.h5ad")
                dataset_handles[handle]["models"][model_type][model_name]["latent_dir"][n_neighbors] = latent_dir
            if model_type == "tardis":
                targets = dataset_handles[handle]["models"][model_type][model_name]["targets"]
                dataset_handles[handle]["models"][model_type][model_name]["targets_latent"] = {i: list(range(8 * ind, 8 * (ind+1))) for ind, i in enumerate(targets)}
                dataset_handles[handle]["models"][model_type][model_name]["targets_latent"]["unreserved"] = list(range(len(targets) * 8, (3 + len(targets)) * 8 ))

In [6]:
args_list = []

for handle in dataset_handles:
    for model_type in dataset_handles[handle]["models"]:
        for model_name in dataset_handles[handle]["models"][model_type]:
            for n_neighbors in dataset_handles[handle]["models"][model_type][model_name]["latent_dir"]:
                if model_type == "tardis":
                    _helper = [(k, v) for k, v in dataset_handles[handle]["models"][model_type][model_name]["targets_latent"].items()]
                
                args_list.append([
                    dataset_handles[handle]["adata_dir"],  # adata_dir, 
                    dataset_handles[handle]["models"][model_type][model_name]["model_dir"],  # model_dir, 
                    dataset_handles[handle]["models"][model_type][model_name]["latent_dir"][n_neighbors],  # latent_dir, 
                    str(n_neighbors),  # n_neighbors, 
                    model_name, # model_name, 
                    model_type,  # model_type
                    "placeholder" if model_type != "tardis" else ".".join([k for k, v in _helper]),  # target_ids
                    "placeholder" if model_type != "tardis" else ".".join([','.join([str(i) for i in v]) for k, v in _helper]),  # target_reserves
                ])

In [8]:
for args in args_list:
    data_dir, model_dir, output_dir, n_neighbors, model_name, model_type, target_ids, target_reserves = args
    print(model_type, model_name, n_neighbors)

    # Check if output directory does not exist and model directory does exist
    if os.path.exists(output_dir):
        print(f"Latent is calculated already: `{model_name, n_neighbors}`")
    
    elif not os.path.exists(model_dir):
        print(f"Model is not there: `{model_name}`")
    
    elif not os.path.exists(output_dir) and os.path.exists(model_dir):
        # Generate the Slurm script
        slurm_script = f"""#!/bin/bash

#SBATCH -J tardis_latent_run
#SBATCH -p gpu_p
#SBATCH --qos=gpu_normal
#SBATCH --gres=gpu:1
#SBATCH -c 6
#SBATCH --mem=159G
#SBATCH --nice=0
#SBATCH -t 1-23:50:00

source activate tardis_env
python /home/icb/kemal.inecik/work/codes/tardis/notebooks/figure2c/collect_latent.py {data_dir} {model_dir} {output_dir} {n_neighbors}
"""

        # Save the script to a temporary file
        script_file = os.path.join(os.getcwd(), f"slurm_job_{model_name}_{n_neighbors}.sh")
        with open(script_file, "w") as file:
            file.write(slurm_script)

        # Submit the Slurm job
        subprocess.run(["sbatch", script_file])
        # subprocess.run(["rm", "-rf", script_file])

    else:
        raise ValueError

scvi kanemaru 15
Latent is calculated already: `('kanemaru', '15')`
scvi kanemaru 30
Latent is calculated already: `('kanemaru', '30')`
tardis kanemaru_sex_age_unit 15
Latent is calculated already: `('kanemaru_sex_age_unit', '15')`
tardis kanemaru_sex_age_unit 30
Latent is calculated already: `('kanemaru_sex_age_unit', '30')`
scvi braun 15
Model is not there: `braun`
scvi braun 30
Model is not there: `braun`
tardis braun_sex_age 15
Model is not there: `braun_sex_age`
tardis braun_sex_age 30
Model is not there: `braun_sex_age`
scvi suo 15
Latent is calculated already: `('suo', '15')`
scvi suo 30
Latent is calculated already: `('suo', '30')`
tardis suo_age 15
Latent is calculated already: `('suo_age', '15')`
tardis suo_age 30
Latent is calculated already: `('suo_age', '30')`
tardis suo_sex 15
Latent is calculated already: `('suo_sex', '15')`
tardis suo_sex 30
Latent is calculated already: `('suo_sex', '30')`
tardis suo_organ 15
Latent is calculated already: `('suo_organ', '15')`
tardis s

In [9]:
for args in args_list:
    data_dir, model_dir, latent_dir, n_neighbors, model_name, model_type, target_ids, target_reserves = args
    
    if model_type != "tardis":
        continue
    
    print(model_type, model_name, n_neighbors)

    # Check if output directory does not exist and model directory does exist
    if not os.path.exists(latent_dir):
        print(f"Latent is not yet calculated: `{model_name, n_neighbors}`")
        continue

    a_ = ad.read_h5ad(latent_dir, backed='r')
    if "tardis_subsets" in a_.uns:
        print(f"sublatents are already calculated: `{model_name, n_neighbors}`")
    
    else:
        # Generate the Slurm script
        slurm_script = f"""#!/bin/bash

#SBATCH -J tardis_latent_run2
#SBATCH -p gpu_p
#SBATCH --qos=gpu_normal
#SBATCH --gres=gpu:1
#SBATCH -c 6
#SBATCH --mem=99G
#SBATCH --nice=0
#SBATCH -t 1-23:50:00

source activate tardis_env
python /home/icb/kemal.inecik/work/codes/tardis/notebooks/figure2c/append_sublatents.py {latent_dir} {target_ids} {target_reserves} {n_neighbors}
"""

        # Save the script to a temporary file
        script_file = os.path.join(os.getcwd(), f"slurm_job_{model_name}_{n_neighbors}_append.sh")
        with open(script_file, "w") as file:
            file.write(slurm_script)

        # Submit the Slurm job
        subprocess.run(["sbatch", script_file])
        # subprocess.run(["rm", "-rf", script_file])
    
    del a_
    gc.collect()

tardis kanemaru_sex_age_unit 15
sublatents are already calculated: `('kanemaru_sex_age_unit', '15')`
tardis kanemaru_sex_age_unit 30
sublatents are already calculated: `('kanemaru_sex_age_unit', '30')`
tardis braun_sex_age 15
Latent is not yet calculated: `('braun_sex_age', '15')`
tardis braun_sex_age 30
Latent is not yet calculated: `('braun_sex_age', '30')`
tardis suo_age 15
sublatents are already calculated: `('suo_age', '15')`
tardis suo_age 30
sublatents are already calculated: `('suo_age', '30')`
tardis suo_sex 15
sublatents are already calculated: `('suo_sex', '15')`
tardis suo_sex 30
sublatents are already calculated: `('suo_sex', '30')`
tardis suo_organ 15
sublatents are already calculated: `('suo_organ', '15')`
tardis suo_organ 30
Submitted batch job 20189496
tardis suo_platform 15
sublatents are already calculated: `('suo_platform', '15')`
tardis suo_platform 30
sublatents are already calculated: `('suo_platform', '30')`
tardis suo_sex_age_organ_platform 15
Latent is not yet

In [10]:
# for i in args_list:
#     print(os.path.split(i[2])[1])

In [11]:
# # !ls -lh /lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/*_subset-*.h5ad
# !rm -rf /lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/*_subset-*.h5ad

In [12]:
# !ls -lh /lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/

In [13]:
harmony_training = list({i[0] for i in args_list})
for harmony_training_adata in harmony_training:
    file_name = os.path.split(harmony_training_adata)[1]
    file_name_raw = os.path.splitext(file_name)[0]
    
    print(file_name_raw)
    a_ = ad.read_h5ad(harmony_training_adata, backed='r')

    if 'harmony' in a_.obsm.keys():
        print(f"Harmony already calculated: `{file_name_raw}`")

    else:
        # Generate the Slurm script
        slurm_script = f"""#!/bin/bash

#SBATCH -J tardis_harmony
#SBATCH -p cpu_p
#SBATCH --qos cpu_normal
#SBATCH -c 8
#SBATCH --mem=120G
#SBATCH --nice=0
#SBATCH -t 1-23:50:00

source activate tardis_env
python /home/icb/kemal.inecik/work/codes/tardis/notebooks/figure2c/integrate_harmony.py {harmony_training_adata}
"""

        # Save the script to a temporary file
        script_file = os.path.join(os.getcwd(), f"slurm_job_{file_name_raw}_harmony.sh")
        with open(script_file, "w") as file:
            file.write(slurm_script)

        # Submit the Slurm job
        subprocess.run(["sbatch", script_file])
        # subprocess.run(["rm", "-rf", script_file])
    
    del a_
    gc.collect()

dataset_complete_Braun
Harmony already calculated: `dataset_complete_Braun`
dataset_complete_Kanemaru
Harmony already calculated: `dataset_complete_Kanemaru`
dataset_complete_Miller
Harmony already calculated: `dataset_complete_Miller`
dataset_complete_He
Harmony already calculated: `dataset_complete_He`
dataset_complete_Garcia
Harmony already calculated: `dataset_complete_Garcia`
dataset_complete_Suo
Harmony already calculated: `dataset_complete_Suo`


In [9]:
only_neighbour = {
    30: "bioconservation",
    15: "batchcorrection"
}

for args in args_list:
    data_dir, model_dir, latent_dir, n_neighbors, model_name, model_type, target_ids, target_reserves = args

    metrics = only_neighbour[int(n_neighbors)]
    print(model_type, model_name, n_neighbors, metrics)
    # do it to prevent clash, then combine the calculations in the next cells 

    p1, p2 = os.path.splitext(latent_dir)
    p = p1 + f"_scib_{metrics}.pickle"
    
    # Check if output directory does not exist and model directory does exist
    if not os.path.exists(latent_dir):
        print(f"Latent is not yet calculated: `{model_name, metrics}`")
        continue

    if model_type != "scvi" and "unreserved" not in target_ids:
        print(f"tardis latent does not have unreserved: `{model_name, metrics}`")
        continue

    if os.path.exists(p):
        print(f"metrics already calculated: `{model_name, metrics}`")
        _ = pd.read_pickle(p)
    
    else:
        # Generate the Slurm script
        slurm_script = f"""#!/bin/bash

#SBATCH -J tardis_scib
#SBATCH -p cpu_p
#SBATCH --qos cpu_normal
#SBATCH -c 32
#SBATCH --mem=180G
#SBATCH --nice=0
#SBATCH -t 2-23:50:00

source activate tardis_env
python /home/icb/kemal.inecik/work/codes/tardis/notebooks/figure2c/calculate_scib.py {latent_dir} {model_type} {metrics} {target_ids} {target_reserves} {data_dir}
"""

        # Save the script to a temporary file
        script_file = os.path.join(os.getcwd(), f"slurm_job_{model_name}_{n_neighbors}_scib.sh")
        with open(script_file, "w") as file:
            file.write(slurm_script)

        # Submit the Slurm job
        subprocess.run(["sbatch", script_file])
        # subprocess.run(["rm", "-rf", script_file])


scvi kanemaru 15 batchcorrection
metrics already calculated: `('kanemaru', 'batchcorrection')`
scvi kanemaru 30 bioconservation
metrics already calculated: `('kanemaru', 'bioconservation')`
tardis kanemaru_sex_age_unit 15 batchcorrection
metrics already calculated: `('kanemaru_sex_age_unit', 'batchcorrection')`
tardis kanemaru_sex_age_unit 30 bioconservation
metrics already calculated: `('kanemaru_sex_age_unit', 'bioconservation')`
scvi braun 15 batchcorrection
Latent is not yet calculated: `('braun', 'batchcorrection')`
scvi braun 30 bioconservation
Latent is not yet calculated: `('braun', 'bioconservation')`
tardis braun_sex_age 15 batchcorrection
Latent is not yet calculated: `('braun_sex_age', 'batchcorrection')`
tardis braun_sex_age 30 bioconservation
Latent is not yet calculated: `('braun_sex_age', 'bioconservation')`
scvi suo 15 batchcorrection
metrics already calculated: `('suo', 'batchcorrection')`
scvi suo 30 bioconservation
Submitted batch job 20189583
tardis suo_age 15 batc

calculate scib metrics:
create a sql database with the metrics

latent_dir, batch_key, label_key, metric

- scvi
- tardis
    - unreserved bioconservation and batch-correction (cell-type)
    - reserved batch

In [7]:
!ls /lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/*.pickle

/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/garcia_scvi_15_latent_scib_batchcorrection.pickle
/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/garcia_scvi_30_latent_scib_bioconservation.pickle
/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/garcia_sex_age_status_tardis_15_latent_scib_batchcorrection.pickle
/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/garcia_sex_age_status_tardis_30_latent_scib_bioconservation.pickle
/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/he_scvi_15_latent_scib_batchcorrection.pickle
/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/he_scvi_30_latent_scib_bioconservation.pickle
/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/he_sex_age_tardis_15_latent_scib_batchcorrection.pickle
/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/he_sex_age_tardis_30_latent_scib_bioconservation.pickle
/lustre/groups/ml01/worksp

In [66]:
pd.concat([pd.read_pickle("/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/miller_scvi_30_latent_scib_bioconservation.pickle"), 
           pd.read_pickle("/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/miller_scvi_15_latent_scib_batchcorrection.pickle")], axis=1)

,isolated_labels,nmi_ari_cluster_labels_kmeans_nmi,nmi_ari_cluster_labels_kmeans_ari,silhouette_label,clisi_knn,Bio conservation,silhouette_batch,ilisi_knn,kbet_per_label,graph_connectivity,pcr_comparison,Batch correction
Embedding,,,,,,,,,,,,
X_pca,0.583141,0.636801,0.502131,0.56039,0.998792,0.656251,0.891507,0.038158,0.191535,0.929672,0.0,0.410174
harmony,0.532204,0.499219,0.310881,0.546748,0.992545,0.57632,0.910659,0.147961,0.587375,0.879822,0.565646,0.618293
scvi,0.539946,0.327286,0.215726,0.514679,0.892599,0.498047,0.918135,0.366563,0.771434,0.93773,0.91798,0.782368
Metric Type,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Aggregate score,Batch correction,Batch correction,Batch correction,Batch correction,Batch correction,Aggregate score


In [67]:
pd.concat([pd.read_pickle("/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/miller_age_tardis_30_latent_scib_bioconservation.pickle"), 
           pd.read_pickle("/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/miller_age_tardis_15_latent_scib_batchcorrection.pickle")], axis=1)

,isolated_labels,nmi_ari_cluster_labels_kmeans_nmi,nmi_ari_cluster_labels_kmeans_ari,silhouette_label,clisi_knn,Bio conservation,silhouette_batch,ilisi_knn,kbet_per_label,graph_connectivity,pcr_comparison,Batch correction
Embedding,,,,,,,,,,,,
tardis,0.561809,0.322237,0.200641,0.512118,0.877408,0.494843,0.913288,0.389849,0.736202,0.909745,0.940759,0.777969
Metric Type,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Aggregate score,Batch correction,Batch correction,Batch correction,Batch correction,Batch correction,Aggregate score


In [68]:
pd.concat([pd.read_pickle("/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/miller_sex_tardis_30_latent_scib_bioconservation.pickle"), 
           pd.read_pickle("/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/miller_sex_tardis_15_latent_scib_batchcorrection.pickle")], axis=1)

,isolated_labels,nmi_ari_cluster_labels_kmeans_nmi,nmi_ari_cluster_labels_kmeans_ari,silhouette_label,clisi_knn,Bio conservation,silhouette_batch,ilisi_knn,kbet_per_label,graph_connectivity,pcr_comparison,Batch correction
Embedding,,,,,,,,,,,,
tardis,0.540132,0.302088,0.184612,0.510787,0.877539,0.483032,0.912833,0.397444,0.743258,0.911628,0.932079,0.779449
Metric Type,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Aggregate score,Batch correction,Batch correction,Batch correction,Batch correction,Batch correction,Aggregate score


In [69]:
pd.concat([pd.read_pickle("/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/miller_sex_age_tardis_30_latent_scib_bioconservation.pickle"), 
           pd.read_pickle("/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/miller_sex_age_tardis_15_latent_scib_batchcorrection.pickle")], axis=1)

,isolated_labels,nmi_ari_cluster_labels_kmeans_nmi,nmi_ari_cluster_labels_kmeans_ari,silhouette_label,clisi_knn,Bio conservation,silhouette_batch,ilisi_knn,kbet_per_label,graph_connectivity,pcr_comparison,Batch correction
Embedding,,,,,,,,,,,,
tardis,0.571453,0.313354,0.180254,0.508722,0.87262,0.48928,0.912584,0.408206,0.737595,0.907832,0.940604,0.781364
Metric Type,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Aggregate score,Batch correction,Batch correction,Batch correction,Batch correction,Batch correction,Aggregate score


In [ ]:
latent = ad.read_h5ad("/lustre/groups/ml01/workspace/kemal.inecik/tardis_data/temp/latent/he_sex_age_tardis_30_latent.h5ad")
print(latent.shape)
print(latent.uns["metrics"])
latent.obs["age_cont"] = latent.obs["age"].astype(float)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    sc.pl.umap(
        latent,
        color=[
            "age",
            "age_cont",
            "sex",
            "integration_sample_status",
            "LVL2",
            "LVL1",
            "LVL0",
            "cell_type",
            "concatenated_integration_covariates",
        ],
        ncols=3,
        color_map="inferno",
        frameon=False,
        legend_fontsize="xx-small",
        show=False,
    )
    plt.show()
    del latent
    gc.collect()